# AI Summary Evaluator
## Automatic Grading System for Student Lecture Summaries

**Author:** J Ganesh Kumar Reddy 23bds024  
**Purpose:** Evaluate student-written summaries against lecture transcript using AI  
**Date:** 2025-11-13

---

### System Overview:
This notebook implements an automated evaluation system that:
1. Loads lecture transcript and student summaries
2. Uses Sentence Transformers for semantic similarity analysis
3. Evaluates summaries on multiple parameters (Coverage, Relevance, Clarity, Coherence, Grammar)
4. Generates scores out of 10 with detailed explanations
5. Exports results to Excel automatically

## Step 1: Import Required Libraries
Installing and importing all necessary dependencies

In [1]:
# Install required packages (run once)
import sys
import subprocess

packages = [
    'pandas',
    'numpy',
    'openpyxl',
    'python-docx',
    'sentence-transformers',
    'nltk',
    'textstat',
    'scikit-learn',
    'torch'
]

print("Installing required packages...")
for package in packages:
    print(f"  Installing {package}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', package])

print("✓ All packages installed successfully!")

Installing required packages...
  Installing pandas...
  Installing numpy...
  Installing openpyxl...
  Installing python-docx...
  Installing sentence-transformers...
  Installing nltk...
  Installing textstat...
  Installing scikit-learn...
  Installing torch...
✓ All packages installed successfully!


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from docx import Document
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.tokenize import sent_tokenize
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings
from datetime import datetime
import os

warnings.filterwarnings('ignore')

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

print("✓ All libraries imported successfully!")
print(f"✓ Current working directory: {os.getcwd()}")

c:\Users\RAVIPRAKASH\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Load AI Model
Loading the Sentence Transformer model for semantic similarity analysis

In [ ]:
# Load pre-trained Sentence Transformer model
print("Loading Sentence Transformer model...")
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
print("✓ Model loaded successfully!")
print(f"  Model: paraphrase-MiniLM-L6-v2")
print(f"  Embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading Sentence Transformer model...
✓ Model loaded successfully!
  Model: paraphrase-MiniLM-L6-v2
  Embedding dimension: 384


## Step 3: Load Input Files
Reading lecture transcript and student summaries

In [ ]:
def load_transcript(file_path):
    """
    Load lecture transcript from DOCX file
    """
    try:
        doc = Document(file_path)
        transcript = "\n".join([paragraph.text for paragraph in doc.paragraphs if paragraph.text.strip()])
        print(f"✓ Transcript loaded: {len(transcript)} characters")
        return transcript
    except Exception as e:
        print(f"✗ Error loading transcript: {e}")
        return None

def load_summaries(file_path):
    """
    Load student summaries from Excel file
    """
    try:
        df = pd.read_excel(file_path)
        print(f"✓ Summaries loaded: {len(df)} students")
        print(f"  Columns: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"✗ Error loading summaries: {e}")
        return None

# Download NLTK data with proper error handling
print("\n=== Setting up NLTK ===")
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)  # New tokenizer format

# Load files
print("\n=== Loading Input Files ===")
transcript = load_transcript('transcript.docx')
summaries_df = load_summaries('summary.xlsx')

if transcript and summaries_df is not None:
    print("\n✓ All files loaded successfully!")
    print(f"\nPreview of loaded data:")
    print(summaries_df.head(3))


=== Setting up NLTK ===

=== Loading Input Files ===
✓ Transcript loaded: 61243 characters
✓ Summaries loaded: 82 students
  Columns: ['Timestamp', 'Email Address', 'Name of the Student', 'Roll number', 'Institute', 'Summary(in 100words)']

✓ All files loaded successfully!

Preview of loaded data:
                Timestamp           Email Address     Name of the Student  \
0 2025-08-07 19:57:08.844   cs22b1027@iiitr.ac.in                 Deepthi   
1 2025-08-07 19:57:23.709  23bds014@iiitdwd.ac.in            Bongu Ashish   
2 2025-08-07 19:58:19.742   cs22b1036@iiitr.ac.in  Mudavath Srinivas Naik   

  Roll number     Institute                               Summary(in 100words)  
0   Cs22b1027  IIIT Raichur  Learned about neural networks in AI generative...  
1    23bds014  IIIT Dharwad                                         Insightful  
2   Cs22b1036  IIIT Raichur  In today’s class, I learned about the Med Agen...  


## Step 4: Define Evaluation Functions
Implementing multiple evaluation criteria

In [ ]:
def extract_key_sentences(text, top_n=10):
    """
    Extract key sentences from text using TF-IDF

    Args:
        text: Input text
        top_n: Number of key sentences to extract

    Returns:
        list: Key sentences
    """
    sentences = sent_tokenize(text)
    if len(sentences) <= top_n:
        return sentences

    # Use TF-IDF to find important sentences
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
    top_indices = scores.argsort()[-top_n:][::-1]

    return [sentences[i] for i in sorted(top_indices)]

def evaluate_coverage(summary, transcript, model):
    """
    Evaluate how much important content from transcript is captured
    Uses semantic similarity with key sentences

    Returns:
        float: Score 0-2
    """
    # Extract key sentences from transcript
    key_sentences = extract_key_sentences(transcript, top_n=15)

    # Encode summary and key sentences
    summary_embedding = model.encode(summary, convert_to_tensor=True)
    key_embeddings = model.encode(key_sentences, convert_to_tensor=True)

    # Calculate similarities
    similarities = util.cos_sim(summary_embedding, key_embeddings)[0]

    # Score based on coverage of key points
    high_similarity_count = (similarities > 0.4).sum().item()
    coverage_ratio = high_similarity_count / len(key_sentences)

    return min(2.0, coverage_ratio * 2.5)

def evaluate_relevance(summary, transcript, model):
    """
    Evaluate how relevant the summary is to the transcript
    Uses overall semantic similarity

    Returns:
        float: Score 0-2
    """
    # Encode both texts
    summary_embedding = model.encode(summary, convert_to_tensor=True)
    transcript_embedding = model.encode(transcript[:5000], convert_to_tensor=True)  # Limit for efficiency

    # Calculate semantic similarity
    similarity = util.cos_sim(summary_embedding, transcript_embedding)[0][0].item()

    # Convert to score
    return min(2.0, similarity * 2.5)

def evaluate_clarity(summary):
    """
    Evaluate readability and flow of the summary
    Uses Flesch Reading Ease score

    Returns:
        float: Score 0-2
    """
    try:
        # Flesch Reading Ease (higher = easier to read)
        reading_ease = textstat.flesch_reading_ease(summary)

        # Convert to 0-2 scale (60-100 is good readability)
        if reading_ease >= 60:
            return 2.0
        elif reading_ease >= 40:
            return 1.5
        elif reading_ease >= 20:
            return 1.0
        else:
            return 0.5
    except:
        return 1.0  # Default if calculation fails

def evaluate_coherence(summary):
    """
    Evaluate logical structure and flow
    Checks sentence count, transitions, and structure

    Returns:
        float: Score 0-2
    """
    sentences = sent_tokenize(summary)
    score = 0.0

    # Check sentence count (3-8 is good for 100 word summary)
    if 3 <= len(sentences) <= 8:
        score += 1.0
    elif len(sentences) >= 2:
        score += 0.5

    # Check for transition words
    transition_words = ['however', 'therefore', 'additionally', 'furthermore', 'moreover',
                       'consequently', 'thus', 'also', 'first', 'second', 'finally', 'next']
    summary_lower = summary.lower()
    transitions_found = sum(1 for word in transition_words if word in summary_lower)

    if transitions_found >= 2:
        score += 1.0
    elif transitions_found >= 1:
        score += 0.5

    return min(2.0, score)

def evaluate_grammar(summary):
    """
    Evaluate grammar and language correctness
    Basic checks for common issues

    Returns:
        float: Score 0-2
    """
    score = 2.0

    # Check for multiple spaces
    if re.search(r'\s{2,}', summary):
        score -= 0.2

    # Check for proper sentence capitalization
    sentences = sent_tokenize(summary)
    uncapitalized = sum(1 for s in sentences if s and not s[0].isupper())
    score -= uncapitalized * 0.2

    # Check for sentence ending punctuation
    no_ending = sum(1 for s in sentences if s and s[-1] not in '.!?')
    score -= no_ending * 0.2

    # Check spelling/grammar quality using textstat
    try:
        avg_word_length = textstat.avg_character_per_word(summary)
        if avg_word_length < 3:  # Very short words might indicate issues
            score -= 0.3
    except:
        pass

    return max(0.5, min(2.0, score))

print("✓ Evaluation functions defined successfully!")

✓ Evaluation functions defined successfully!


## Step 5: Main Evaluation Function
Combining all criteria and generating explanations

In [ ]:
def evaluate_summary(summary, transcript, model):
    """
    Comprehensive evaluation of a student summary

    Args:
        summary: Student's summary text
        transcript: Original lecture transcript
        model: Sentence Transformer model

    Returns:
        dict: Score and detailed evaluation
    """
    # Handle missing or invalid summaries
    if not summary or not isinstance(summary, str) or len(summary.strip()) < 20:
        return {
            'score': 0.0,
            'coverage': 0.0,
            'relevance': 0.0,
            'clarity': 0.0,
            'coherence': 0.0,
            'grammar': 0.0,
            'explanation': 'Invalid or missing summary provided.'
        }

    # Evaluate on each criterion
    coverage_score = evaluate_coverage(summary, transcript, model)
    relevance_score = evaluate_relevance(summary, transcript, model)
    clarity_score = evaluate_clarity(summary)
    coherence_score = evaluate_coherence(summary)
    grammar_score = evaluate_grammar(summary)

    # Calculate total score (each criterion worth 2 points, total 10)
    total_score = coverage_score + relevance_score + clarity_score + coherence_score + grammar_score

    # Generate explanation
    explanation_parts = []

    # Coverage feedback
    if coverage_score >= 1.5:
        explanation_parts.append("Excellent coverage of key lecture points")
    elif coverage_score >= 1.0:
        explanation_parts.append("Good coverage but misses some important details")
    else:
        explanation_parts.append("Limited coverage of key lecture content")

    # Relevance feedback
    if relevance_score >= 1.5:
        explanation_parts.append("highly relevant to transcript")
    elif relevance_score >= 1.0:
        explanation_parts.append("mostly relevant with some tangential content")
    else:
        explanation_parts.append("lacks relevance to main lecture themes")

    # Writing quality feedback
    if clarity_score >= 1.5 and grammar_score >= 1.5:
        explanation_parts.append("Well-written with clear language and good grammar")
    elif clarity_score >= 1.0 or grammar_score >= 1.0:
        explanation_parts.append("Adequate writing quality with minor issues")
    else:
        explanation_parts.append("Needs improvement in clarity and grammar")

    # Coherence feedback
    if coherence_score < 1.0:
        explanation_parts.append("Could improve logical flow and structure")

    explanation = ". ".join(explanation_parts) + "."

    return {
        'score': round(total_score, 2),
        'coverage': round(coverage_score, 2),
        'relevance': round(relevance_score, 2),
        'clarity': round(clarity_score, 2),
        'coherence': round(coherence_score, 2),
        'grammar': round(grammar_score, 2),
        'explanation': explanation
    }

print("✓ Main evaluation function ready!")

✓ Main evaluation function ready!


## Step 6: Batch Evaluation Process
Evaluating all student summaries with progress tracking

In [ ]:
def evaluate_all_summaries(summaries_df, transcript, model):
    """
    Evaluate all student summaries

    Args:
        summaries_df: DataFrame with student summaries
        transcript: Lecture transcript
        model: Sentence Transformer model

    Returns:
        DataFrame: Results with scores and explanations
    """
    results = []
    total_students = len(summaries_df)

    print(f"\n{'='*60}")
    print(f"Starting evaluation of {total_students} student summaries...")
    print(f"{'='*60}\n")

    # Identify the summary column (handle different possible names)
    summary_col = None
    for col in summaries_df.columns:
        if 'summary' in col.lower():
            summary_col = col
            break

    if summary_col is None:
        print("✗ Error: Could not find summary column!")
        return None

    for idx, row in summaries_df.iterrows():
        student_num = idx + 1
        print(f"[{student_num}/{total_students}] Evaluating: {row.get('Name of the Student', 'Unknown')}")

        # Get summary text
        summary_text = row[summary_col]

        # Evaluate
        evaluation = evaluate_summary(summary_text, transcript, model)

        # Store results
        result = {
            'Email Address': row.get('Email Address', ''),
            'Name': row.get('Name of the Student', ''),
            'Roll Number': row.get('Roll number', ''),
            'Institute': row.get('Institute', ''),
            'Summary': summary_text,
            'Score': evaluation['score'],
            'Coverage': evaluation['coverage'],
            'Relevance': evaluation['relevance'],
            'Clarity': evaluation['clarity'],
            'Coherence': evaluation['coherence'],
            'Grammar': evaluation['grammar'],
            'Explanation': evaluation['explanation']
        }
        results.append(result)

        print(f"  → Score: {evaluation['score']}/10")
        print()

    print(f"{'='*60}")
    print(f"✓ Evaluation completed for all {total_students} students!")
    print(f"{'='*60}\n")

    return pd.DataFrame(results)

# Run evaluation
if transcript and summaries_df is not None:
    results_df = evaluate_all_summaries(summaries_df, transcript, model)
else:
    print("✗ Cannot proceed: Files not loaded properly")


Starting evaluation of 82 student summaries...

[1/82] Evaluating: Deepthi
  → Score: 2.85/10

[2/82] Evaluating: Bongu Ashish
  → Score: 0.0/10

[3/82] Evaluating: Mudavath Srinivas Naik
  → Score: 5.01/10

[4/82] Evaluating: Aman Chaurasia
  → Score: 5.09/10

[5/82] Evaluating: Krishu Patel
  → Score: 4.61/10

[6/82] Evaluating: Mohammed Arsalan 
  → Score: 4.25/10

[7/82] Evaluating: Mohammed Nabeel Ahsan
  → Score: 3.74/10

[8/82] Evaluating: MODINE Karthik
  → Score: 5.76/10

[9/82] Evaluating: Chidvilasini Sridharala 
  → Score: 4.86/10

[10/82] Evaluating: M Jagadeeswar Reddy
  → Score: 6.59/10

[11/82] Evaluating: Jyoti Gadad
  → Score: 5.88/10

[12/82] Evaluating: Prasad K
  → Score: 4.51/10

[13/82] Evaluating: Pavan Kumar
  → Score: 5.38/10

[14/82] Evaluating: Aditya Gupta 
  → Score: 6.17/10

[15/82] Evaluating: Ritik Kumar Shahi 
  → Score: 5.42/10

[16/82] Evaluating: Pratyush Kumar 
  → Score: 5.72/10

[17/82] Evaluating: A.M.D.Pradeep
  → Score: 4.63/10

[18/82] Evalu

## Step 7: Display Results
Showing evaluation statistics and top results

In [ ]:
if results_df is not None:
    print("\n" + "="*80)
    print(" "*25 + "EVALUATION STATISTICS")
    print("="*80 + "\n")

    print(f"Total Students Evaluated: {len(results_df)}")
    print(f"Average Score: {results_df['Score'].mean():.2f}/10")
    print(f"Highest Score: {results_df['Score'].max():.2f}/10")
    print(f"Lowest Score: {results_df['Score'].min():.2f}/10")
    print(f"Standard Deviation: {results_df['Score'].std():.2f}")

    print("\n" + "-"*80)
    print("Average Scores by Criterion:")
    print("-"*80)
    print(f"  Coverage (0-2):   {results_df['Coverage'].mean():.2f}")
    print(f"  Relevance (0-2):  {results_df['Relevance'].mean():.2f}")
    print(f"  Clarity (0-2):    {results_df['Clarity'].mean():.2f}")
    print(f"  Coherence (0-2):  {results_df['Coherence'].mean():.2f}")
    print(f"  Grammar (0-2):    {results_df['Grammar'].mean():.2f}")

    print("\n" + "="*80)
    print(" "*30 + "TOP 5 RESULTS")
    print("="*80 + "\n")

    top_5 = results_df.nlargest(5, 'Score')[['Name', 'Roll Number', 'Score', 'Explanation']]

    for idx, row in top_5.iterrows():
        print(f"Rank {list(top_5.index).index(idx) + 1}:")
        print(f"  Name: {row['Name']}")
        print(f"  Roll: {row['Roll Number']}")
        print(f"  Score: {row['Score']}/10")
        print(f"  Feedback: {row['Explanation']}")
        print("-"*80)

    print("\n✓ Results preview complete!")


                         EVALUATION STATISTICS

Total Students Evaluated: 82
Average Score: 5.16/10
Highest Score: 7.09/10
Lowest Score: 0.00/10
Standard Deviation: 1.12

--------------------------------------------------------------------------------
Average Scores by Criterion:
--------------------------------------------------------------------------------
  Coverage (0-2):   0.69
  Relevance (0-2):  0.40
  Clarity (0-2):    1.21
  Coherence (0-2):  1.07
  Grammar (0-2):    1.79

                              TOP 5 RESULTS

Rank 1:
  Name: Aryan Anilkumar Talikoti 
  Roll: 23bcs018
  Score: 7.09/10
  Feedback: Good coverage but misses some important details. lacks relevance to main lecture themes. Well-written with clear language and good grammar.
--------------------------------------------------------------------------------
Rank 2:
  Name: M Jagadeeswar Reddy
  Roll: 23bds033
  Score: 6.59/10
  Feedback: Excellent coverage of key lecture points. lacks relevance to main lecture t

## Step 8: Export Results to Excel
Saving graded results with formatting

In [ ]:
def export_results(results_df, roll_number='23bds024'):
    """
    Export evaluation results to Excel file

    Args:
        results_df: DataFrame with evaluation results
        roll_number: Student roll number for filename
    """
    output_filename = f'Graded_results_{roll_number}.xlsx'

    try:
        # Sort by score (highest first)
        results_sorted = results_df.sort_values('Score', ascending=False).reset_index(drop=True)

        # Create Excel writer
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            # Write main results
            results_sorted.to_excel(writer, sheet_name='Graded Results', index=False)

            # Get workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets['Graded Results']

            # Adjust column widths
            column_widths = {
                'A': 30,  # Email
                'B': 25,  # Name
                'C': 15,  # Roll Number
                'D': 20,  # Institute
                'E': 50,  # Summary
                'F': 10,  # Score
                'G': 12,  # Coverage
                'H': 12,  # Relevance
                'I': 12,  # Clarity
                'J': 12,  # Coherence
                'K': 12,  # Grammar
                'L': 60   # Explanation
            }

            for col, width in column_widths.items():
                worksheet.column_dimensions[col].width = width

            # Create summary statistics sheet
            stats_data = {
                'Metric': ['Total Students', 'Average Score', 'Highest Score', 'Lowest Score',
                          'Standard Deviation', 'Avg Coverage', 'Avg Relevance', 'Avg Clarity',
                          'Avg Coherence', 'Avg Grammar'],
                'Value': [
                    len(results_sorted),
                    f"{results_sorted['Score'].mean():.2f}/10",
                    f"{results_sorted['Score'].max():.2f}/10",
                    f"{results_sorted['Score'].min():.2f}/10",
                    f"{results_sorted['Score'].std():.2f}",
                    f"{results_sorted['Coverage'].mean():.2f}/2",
                    f"{results_sorted['Relevance'].mean():.2f}/2",
                    f"{results_sorted['Clarity'].mean():.2f}/2",
                    f"{results_sorted['Coherence'].mean():.2f}/2",
                    f"{results_sorted['Grammar'].mean():.2f}/2"
                ]
            }
            stats_df = pd.DataFrame(stats_data)
            stats_df.to_excel(writer, sheet_name='Statistics', index=False)

        print(f"\n{'='*80}")
        print(f" "*25 + "EXPORT SUCCESSFUL")
        print(f"{'='*80}")
        print(f"\n✓ Results exported to: {output_filename}")
        print(f"  Location: {os.path.abspath(output_filename)}")
        print(f"  Total records: {len(results_sorted)}")
        print(f"  Sheets created: 'Graded Results', 'Statistics'")
        print(f"\n{'='*80}\n")

        return output_filename

    except Exception as e:
        print(f"\n✗ Error exporting results: {e}")
        return None

# Export results
if results_df is not None:
    output_file = export_results(results_df, roll_number='23bds024')
else:
    print("✗ No results to export")


                         EXPORT SUCCESSFUL

✓ Results exported to: Graded_results_23bds024.xlsx
  Location: c:\Users\RAVIPRAKASH\Downloads\Agentic AI Quiz\Graded_results_23bds024.xlsx
  Total records: 82
  Sheets created: 'Graded Results', 'Statistics'




## Step 9: Final Summary
System execution summary and demo readiness check

In [26]:
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*20 + "AI SUMMARY EVALUATOR - EXECUTION COMPLETE" + " "*17 + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

print("✓ System successfully completed all tasks:")
print("\n  1. ✓ Loaded lecture transcript from transcript.docx")
print("  2. ✓ Loaded student summaries from summary.xlsx")
print("  3. ✓ Initialized AI model (Sentence Transformer)")
print("  4. ✓ Evaluated all summaries on 5 criteria:")
print("       • Coverage - Key content capture")
print("       • Relevance - Alignment with transcript")
print("       • Clarity - Readability and flow")
print("       • Coherence - Logical structure")
print("       • Grammar - Language correctness")
print("  5. ✓ Generated scores (0-10) with explanations")
print("  6. ✓ Created Excel file: Graded_results_22bds053.xlsx")
print("  7. ✓ Displayed top 5 results for inspection\n")

if results_df is not None:
    print(f"📊 Evaluation Summary:")
    print(f"   • Students evaluated: {len(results_df)}")
    print(f"   • Average score: {results_df['Score'].mean():.2f}/10")
    print(f"   • Score range: {results_df['Score'].min():.2f} - {results_df['Score'].max():.2f}")

print("\n" + "#"*80)
print("#" + " "*25 + "SYSTEM READY FOR DEMO VIDEO" + " "*26 + "#")
print("#"*80 + "\n")

print("📹 Demo Checklist:")
print("   ✓ Input files loaded and processed")
print("   ✓ AI evaluation pipeline functional")
print("   ✓ Results generated with scores and feedback")
print("   ✓ Excel output file created in working directory")
print("   ✓ Top results displayed for verification")
print("\n   Ready to record 5-minute demonstration video!\n")

print("="*80)
print("To re-run: Click 'Run All' or restart kernel and execute all cells")
print("="*80)


################################################################################
#                                                                              #
#                    AI SUMMARY EVALUATOR - EXECUTION COMPLETE                 #
#                                                                              #
################################################################################

✓ System successfully completed all tasks:

  1. ✓ Loaded lecture transcript from transcript.docx
  2. ✓ Loaded student summaries from summary.xlsx
  3. ✓ Initialized AI model (Sentence Transformer)
  4. ✓ Evaluated all summaries on 5 criteria:
       • Coverage - Key content capture
       • Relevance - Alignment with transcript
       • Clarity - Readability and flow
       • Coherence - Logical structure
       • Grammar - Language correctness
  5. ✓ Generated scores (0-10) with explanations
  6. ✓ Created Excel file: Graded_results_22bds053.xlsx
  7. ✓ Displayed top 5 results for in